# Building Agent Systems with Databricks -- HealthVerity Clinical Assistant Demo

## Part 1 - Architect Your First Agent
This agent will follow the workflow of a healthcare provider analyzing patient data to illustrate various agent capabilities using HealthVerity's real-world healthcare dataset. 
We'll assume the provider would be trying to quickly understand a patient's healthcare journey across medical claims, pharmacy data, and procedures to make informed clinical decisions.

### 1.1 Build Simple Tools
- **SQL Functions**: Create queries that access HealthVerity's real-world healthcare data including medical claims, pharmacy claims, diagnoses, and procedures.
- **Simple Python Function**: Create and register a Python function to overcome some common limitations of language models when analyzing healthcare data.

### 1.2 Integrate with an LLM [AI Playground]
- Combine the tools you created with a Language Model (LLM) in the AI Playground.

### 1.3 Test the Agent [AI Playground]
- Ask the agent a question and observe the response.
- Dive deeper into the agent’s performance by exploring MLflow traces.


In [0]:
%pip install -qqqq -r requirements.txt
# Restart to load the packages into the Python environment
dbutils.library.restartPython()

In [0]:
import yaml

with open("config.yaml", "r") as f:
    config = yaml.safe_load(f)

source_catalog_name = config["source_catalog_name"]
source_schema_name = config["source_schema_name"]
target_catalog_name = config["target_catalog_name"]
target_schema_name = config["target_schema_name"]
service_date = config["service_date"]
claim_id = config["claim_id"]
patient_id = config["patient_id"]
diagnosis_code = config["diagnosis_code"]
ndc_code = config["ndc_code"]
model_uc_name = config["model_uc_name"]
alias = config["alias"]
endpoint_name = config["endpoint_name"]

In [0]:
from databricks.sdk import WorkspaceClient
import yaml
import os

w = WorkspaceClient()
workspace_id = str(w.get_workspace_id())

print(f"Source data from: {source_catalog_name}.{source_schema_name}")
print(f"Target functions will be created in: {target_catalog_name}.{target_schema_name}")

In [0]:
dbutils.widgets.removeAll()

In [0]:
# Allows us to reference these values directly in the SQL/Python function creation
dbutils.widgets.text("source_catalog_name", defaultValue=source_catalog_name)
dbutils.widgets.text("source_schema_name", defaultValue=source_schema_name)
dbutils.widgets.text("target_catalog_name", defaultValue=target_catalog_name)
dbutils.widgets.text("target_schema_name", defaultValue=target_schema_name)
dbutils.widgets.text("workspace_id", defaultValue=workspace_id)
dbutils.widgets.text("service_date", defaultValue=service_date)

dbutils.widgets.text("claim_id", defaultValue=claim_id)
dbutils.widgets.text("patient_id", defaultValue=patient_id)
dbutils.widgets.text("diagnosis_code", defaultValue=diagnosis_code)
dbutils.widgets.text("ndc_code", defaultValue=ndc_code)

In [0]:
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {target_catalog_name}.{target_schema_name}")

# HealthVerity Clinical Assistant Workflow

Below is a structured outline of the **key steps** a healthcare provider might follow when **analyzing patient data using HealthVerity's real-world healthcare dataset**. 
---

## 1. Get patient enrollment and demographics
- **Action**: Retrieve patient enrollment information and demographics from the enrollment table
- **Why**: Understanding patient demographics, enrollment periods, and benefit types provides essential context for healthcare analysis.

---

In [0]:
%sql
-- find patient enrollment information given a patient id
select * from IDENTIFIER(:source_catalog_name || '.' || :source_schema_name || '.' || 'enrollment')
where patient_id = :patient_id
order by date_start desc
limit 1;

In [0]:
%sql
-- First lets make sure it doesn't already exist
DROP FUNCTION IF EXISTS IDENTIFIER(:target_catalog_name || '.' || :target_schema_name || '.' || 'get_patient_enrollment');

-- Now we create our first function. This takes in patient id and returns enrollment information.
CREATE OR REPLACE FUNCTION ${target_catalog_name}.${target_schema_name}.get_patient_enrollment(
  patient_id STRING COMMENT 'The patient id'
  )
RETURNS TABLE (
  patient_id STRING, 
  patient_gender STRING, 
  patient_year_of_birth STRING, 
  patient_zip3 STRING, 
  patient_state STRING, 
  date_start DATE, 
  date_end DATE,
  benefit_type STRING,
  pay_type STRING
)
COMMENT 'Returns patient enrollment information including demographics, enrollment dates, and benefit details for a particular patient given their unique patient ID.'
RETURN
(
  SELECT 
    patient_id, 
    patient_gender, 
    patient_year_of_birth, 
    patient_zip3, 
    patient_state, 
    date_start, 
    date_end,
    benefit_type,
    pay_type
  FROM ${source_catalog_name}.${source_schema_name}.enrollment
  WHERE patient_id = patient_id
  ORDER BY date_start DESC
  LIMIT 1
);

In [0]:
%sql
select * from ${target_catalog_name}.${target_schema_name}.get_patient_enrollment(:patient_id)

---

## 2. Retrieve Medical Claims
- **Action**: Access medical claims data for a patient on a specific service date
- **Why**: Medical claims provide detailed information about healthcare services, procedures, and costs for comprehensive patient analysis.

---

In [0]:
%sql
-- First lets make sure it doesn't already exist
DROP FUNCTION IF EXISTS IDENTIFIER(:target_catalog_name || '.' || :target_schema_name || '.' || 'get_medical_claims');

-- Now we create our second function. This takes in patient_id and service_date
CREATE OR REPLACE FUNCTION ${target_catalog_name}.${target_schema_name}.get_medical_claims(
  patient_input_id STRING COMMENT 'The patient id', 
  service_date DATE COMMENT 'The date of service'
  )
RETURNS TABLE (
  claim_id STRING, 
  patient_id STRING, 
  date_service DATE, 
  date_service_end DATE,
  location_of_care STRING,
  pay_type STRING
)
COMMENT 'Returns medical claims information including claim id, patient id, service dates, location of care, and payment type for a particular patient on a specific service date'
RETURN
(
  SELECT 
    claim_id, 
    patient_id, 
    date_service, 
    date_service_end,
    location_of_care,
    pay_type
  FROM ${source_catalog_name}.${source_schema_name}.medical_claim
  WHERE patient_id = patient_input_id and date_service = service_date
);

In [0]:
%sql
select * from ${target_catalog_name}.${target_schema_name}.get_medical_claims(:patient_id, :service_date)

---

## 3. Make Functions for Diagnosis Codes and Pharmacy Claims
- **Action**: get information about diagnosis codes for a patient and retrieve pharmacy claims data
- **Why**: Healthcare providers need to understand patient conditions through diagnosis codes and medication history through pharmacy claims for comprehensive care analysis

In [0]:
%sql
-- first one gets diagnosis information for a specific patient
select patient_id, claim_id, date_service, diagnosis_code, diagnosis_qual, admit_diagnosis_ind
from IDENTIFIER(:source_catalog_name || '.' || :source_schema_name || '.' || 'diagnosis')
where patient_id = :patient_id;

In [0]:
%sql
-- second one returns pharmacy claims for a given patient
select claim_id, patient_id, date_service, ndc, fill_number, days_supply, dispensed_quantity, pay_type, copay_coinsurance
from IDENTIFIER(:source_catalog_name || '.' || :source_schema_name || '.' || 'pharmacy_claim')
where patient_id = :patient_id;

In [0]:
%sql
-- First lets make sure it doesn't already exist
DROP FUNCTION IF EXISTS IDENTIFIER(:target_catalog_name || '.' || :target_schema_name || '.' || 'get_patient_diagnoses');

-- This takes in patient_id to get all diagnosis codes for that patient
CREATE OR REPLACE FUNCTION ${target_catalog_name}.${target_schema_name}.get_patient_diagnoses(
  patient_input_id STRING COMMENT 'The patient id'
  )
RETURNS TABLE (
  claim_id STRING, 
  patient_id STRING, 
  date_service DATE, 
  date_service_end DATE,
  diagnosis_code STRING,
  diagnosis_qual STRING,
  admit_diagnosis_ind STRING
)
COMMENT 'Returns diagnosis information including claim id, patient id, service dates, diagnosis code, qualifier, and admission diagnosis indicator for a specific patient'
RETURN
(
  SELECT 
    claim_id, 
    patient_id, 
    date_service, 
    date_service_end,
    diagnosis_code,
    diagnosis_qual,
    admit_diagnosis_ind
  FROM ${source_catalog_name}.${source_schema_name}.diagnosis
  WHERE patient_id = patient_input_id
  ORDER BY date_service DESC
  LIMIT 50
);

In [0]:
%sql
select * from ${target_catalog_name}.${target_schema_name}.get_patient_diagnoses(:patient_id)

In [0]:
%sql

-- First lets make sure it doesn't already exist
DROP FUNCTION IF EXISTS IDENTIFIER(:target_catalog_name || '.' || :target_schema_name || '.' || 'get_pharmacy_claims');

-- This takes in patient_id to get pharmacy claims
CREATE OR REPLACE FUNCTION ${target_catalog_name}.${target_schema_name}.get_pharmacy_claims(
  patient_input_id STRING COMMENT 'The patient id'
  )
RETURNS TABLE (
  claim_id STRING, 
  patient_id STRING,  
  date_service DATE,
  ndc STRING,
  fill_number INT,
  days_supply INT,
  dispensed_quantity FLOAT,
  pay_type STRING,
  copay_coinsurance FLOAT,
  submitted_gross_due FLOAT,
  paid_gross_due FLOAT
)
COMMENT 'Returns pharmacy claims information including claim details, NDC codes, fill information, and payment details for a given patient. Example: claim_id: 66314b1117ffbdbbbbee693a9f69453d, ndc: 65162027250, days_supply: 30, dispensed_quantity: 1.0'
RETURN
(
  SELECT 
    claim_id, 
    patient_id,  
    date_service,
    ndc,
    fill_number,
    days_supply,
    dispensed_quantity,
    pay_type,
    copay_coinsurance,
    submitted_gross_due,
    paid_gross_due
  FROM ${source_catalog_name}.${source_schema_name}.pharmacy_claim
  WHERE patient_id = patient_input_id
  ORDER BY date_service DESC 
  LIMIT 50
);

In [0]:
%sql
select * from ${target_catalog_name}.${target_schema_name}.get_pharmacy_claims(:patient_id)

-- this will return all pharmacy claims for the patient including medication details and costs 

---

## 4. Create Function for Procedure Analysis
- **Action**: Retrieve procedure codes and details for comprehensive healthcare analysis
- **Why**: Understanding procedures performed helps complete the patient's healthcare story alongside diagnoses and medications

---

In [0]:
%sql

-- First lets make sure it doesn't already exist
DROP FUNCTION IF EXISTS IDENTIFIER(:target_catalog_name || '.' || :target_schema_name || '.' || 'get_patient_procedures');

-- This takes in patient_id to get procedure information
CREATE OR REPLACE FUNCTION ${target_catalog_name}.${target_schema_name}.get_patient_procedures(
  patient_input_id STRING COMMENT 'The patient id'
  )
RETURNS TABLE (
  claim_id STRING, 
  patient_id STRING,
  service_line_number STRING,
  date_service DATE,
  date_service_end DATE,
  procedure_code STRING,
  procedure_qual STRING,
  procedure_units FLOAT,
  revenue_code STRING,
  line_charge FLOAT,
  line_allowed FLOAT
)
COMMENT 'Returns procedure information including claim details, procedure codes, service dates, units, and charges for a given patient'
RETURN
(
  SELECT 
    claim_id, 
    patient_id,
    service_line_number,
    date_service,
    date_service_end,
    procedure_code,
    procedure_qual,
    procedure_units,
    revenue_code,
    line_charge,
    line_allowed
  FROM ${source_catalog_name}.${source_schema_name}.procedure
  WHERE patient_id = patient_input_id
  ORDER BY date_service DESC 
  LIMIT 50
);

In [0]:
%sql
select * from ${target_catalog_name}.${target_schema_name}.get_patient_procedures(:patient_id)

In [0]:
from IPython.display import display, HTML

# Retrieve the Databricks host URL
workspace_url = spark.conf.get('spark.databricks.workspaceUrl')

# Create HTML link to created functions
html_link = f'<a href="https://{workspace_url}/explore/data/functions/{target_catalog_name}/{target_schema_name}/get_patient_enrollment" target="_blank">Go to Unity Catalog to see Registered Functions</a>'
display(HTML(html_link))

## Now lets go over to the AI Playground to see how we can use these functions and assemble our first HealthVerity Agent!

### The AI Playground can be found on the left navigation bar under 'Machine Learning' or you can use the link created below

Example questions to try:
- `What is the healthcare journey for patient 4baf3314e4a181c5effcf2751fbe1e21?`
- `What medications has patient 4baf3314e4a181c5effcf2751fbe1e21 been prescribed?`
- `What are the diagnosis codes for patient 4baf3314e4a181c5effcf2751fbe1e21?`

In [0]:
# Create HTML link to AI Playground
html_link = f'<a href="https://{workspace_url}/ml/playground" target="_blank">Go to AI Playground</a>'
display(HTML(html_link))